In [ ]:
%pip install -Uq sagemaker docx2pdf

In [ ]:
%pip install -q langchain_community langgraph langchain-aws

In [ ]:
%pip install -q python-docx

In [ ]:
import os
import json
# import dotenv


os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "comics"
os.environ["api_endpoint"] =  "http://127.0.0.1:8080/invocations"
#load env from .env
# dotenv.load_dotenv('./.env')

## 1. Utils functions
### 1.1 Structrued Output parser

In [ ]:
import re
from langchain_core.output_parsers.base import BaseOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from json import JSONDecodeError

def dict_to_obj(json_str:dict, target:object):
    return target.parse_obj(json_str)


class CustJsonOuputParser(BaseOutputParser[str]): 
    verbose :bool = Field( default=True)

    def parse(self, text: str) -> str:
        if self.verbose:
            print(text)
        pattern = r"<answer>(.*?)</answer>"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            text = match.group(1)
            # print(text)
            # text = text.replace('[','(').replace(']',')') ##避免跟sentiment的格式冲突
        else:
            return {'answer':"no"}    
        new_dict = json.loads(text.replace('\n','  '))
        
        return new_dict

    @property
    def _type(self) -> str:
        return "cust_output_parser"

class TextOuputParser(BaseOutputParser[str]): 
    verbose :bool = Field( default=True)

    def parse(self, text: str) -> str:
        if self.verbose:
            print(text)
        pattern = r"<answer>(.*?)</answer>"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            text = match.group(1)
            return text.strip()
        else:
            return ''

    @property
    def _type(self) -> str:
        return "TextOuputParser"
    
    
outparser = CustJsonOuputParser(verbose=False)

### 1.2 LLM models

In [ ]:
from langchain_aws import ChatBedrock

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_sonnet = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0.8,
                                "top_k":250,
                                "max_tokens": 4096,
                                "top_p":0.9,
                                "stop_sequences":['</invoke>','</error>']
                               },
                  streaming=True,
                #   callbacks=[StreamingStdOutCallbackHandler()]
                )

llm_haiku = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0.8,
                                "top_k":250,
                                "max_tokens": 4096,
                                "top_p":0.9,
                                "stop_sequences":['</invoke>','</error>']
                               },
                  streaming=True,
                  # callbacks=[StreamingStdOutCallbackHandler()]
                  )

llm_llama = ChatBedrock(model_id="meta.llama3-70b-instruct-v1:0",
                  model_kwargs={"temperature": 0.5,
                                "max_gen_len": 2048,
                                "top_p":0.9,
                                # "stop_sequences":['</invoke>','</error>']
                               },
                  streaming=True,
                  # callbacks=[StreamingStdOutCallbackHandler()]
                  )

llm_cr = ChatBedrock(model_id="cohere.command-r-v1:0",
                  model_kwargs={"temperature": 0.8,
                                "top_k":250,
                                "max_tokens": 4096,
                                "top_p":0.5,
                                "stop_sequences":['</invoke>','</error>']
                               },
                  streaming=True,
                  # callbacks=[StreamingStdOutCallbackHandler()]
                  )

llm = llm_haiku


In [ ]:
# llm_llama.invoke("[USER] hello [USER] ")
# llm_cr.invoke("hello")

### 1.3 Image Generation model invoke code

- Bedrock SD model

In [ ]:
import base64
import io
import json
import boto3
from PIL import Image
from botocore.exceptions import ClientError
from enum import Enum
from io import BytesIO

# profile='default'
profile=''

class StyleEnum(Enum):
    Photographic = "photographic"
    Tile_texture = "tile-texture"
    Digital_art = "digital-art"
    Origami = "origami"
    Modeling_compound = "modeling-compound"
    Anime = "anime"
    Cinematic = "cinematic"
    Model_3D = "3d-model"
    Comicbook = "comic-book"
    Enhance = "enhance"
    
class ImageError(Exception):
    "Custom exception for errors returned by SDXL"
    def __init__(self, message):
        self.message = message


class ImageGenerator(BaseModel):
    model_id: str = Field(default="stability.stable-diffusion-xl-v1")
    profile:str
    cfg_scale: int = Field( default=7)
    steps:int = Field( default=50)
    samples:int = Field( default=1)
    
    def _generate(self,model_id, body):
        """
        Generate an image using SDXL 1.0 on demand.
        Args:
            model_id (str): The model ID to use.
            body (str) : The request body to use.
        Returns:
            image_bytes (bytes): The image generated by the model.
        """

        # logger.info("Generating image with SDXL model %s", model_id)

        session = boto3.Session()
        # session = boto3.Session(profile_name=self.profile)
        #get bedrock service 
        bedrock = session.client(service_name='bedrock-runtime')
    
        accept = "application/json"
        content_type = "application/json"

        response = bedrock.invoke_model(
            body=body, modelId=model_id, accept=accept, contentType=content_type
        )
        response_body = json.loads(response.get("body").read())

        base64_image = response_body.get("artifacts")[0].get("base64")
        base64_bytes = base64_image.encode('ascii')
        image_bytes = base64.b64decode(base64_bytes)

        finish_reason = response_body.get("artifacts")[0].get("finishReason")

        if finish_reason == 'ERROR' or finish_reason == 'CONTENT_FILTERED':
            raise ImageError(f"Image generation error. Error code is {finish_reason}")


        # logger.info("Successfully generated image withvthe SDXL 1.0 model %s", model_id)

        return image_bytes

    def generate_image( self,prompt,seed=0,style_preset=StyleEnum.Photographic.value):
        if self.model_id.startswith('stability'):
            body=json.dumps({
                "text_prompts": [
                {
                "text": prompt
                }
            ],
            "cfg_scale": self.cfg_scale,
            "seed": seed,
            "steps": self.steps,
              "height": 768,
            "width": 768,
            "samples" : self.samples,
            "style_preset" : style_preset
            })
        elif self.model_id.startswith('amazon'):
            body = json.dumps({
            "taskType": "TEXT_IMAGE",
            "textToImageParams": {
                "text": prompt
            },
            "imageGenerationConfig": {
                "numberOfImages": 1,
                "height": 768,
                "width": 768,
                "cfgScale": self.cfg_scale,
                "seed": seed
            }
            })
        print(body)
        image= None
        try:
            image_bytes=self._generate(model_id = self.model_id,
                                    body = body)
            image = Image.open(io.BytesIO(image_bytes))

        except ClientError as err:
            message=err.response["Error"]["Message"]
            # logger.error("A client error occurred: %s", message)
            print("A client error occured: " +format(message))
        except ImageError as err:
            print(err)
        except Exception as err:
            print(err)
        finally:
            return image
        


- Sagemaker model

In [ ]:
import sagemaker
from typing import Any, List
import json
import io
from sagemaker.async_inference.waiter_config import WaiterConfig
import time
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor



def base64_to_image(base64_string):
    image_bytes = base64.b64decode(base64_string)
    image_buffer = BytesIO(image_bytes)
    image = Image.open(image_buffer)
    return image

def get_bucket_and_key(s3uri):
    pos = s3uri.find("/", 5)
    bucket = s3uri[5:pos]
    key = s3uri[pos + 1 :]
    return bucket, key



class StoryDiffusionGenerator():
    """ API schema
    class APIRequest(BaseModel):
        sd_type: Literal['RealVision','SDXL','Unstable'] = Field(default='SDXL')
        modeltype : Literal["Only Using Textual Description","Using Ref Images"] = Field(default="Only Using Textual Description")
        files: Any = Field(default=None)
        num_steps : int = Field(default=50)
        style : Literal["Japanese Anime","(No style)","Cinematic","Disney Charactor","Photographic","Comic book","Line art"] = Field(default="Comic book")
        Ip_Adapter_Strength : float = Field(default=0.5, descrition="The strength of the IP adapter. The value ranges from 0 to 1. The larger the value, the stronger the IP adapter.")
        style_strength_ratio : int = Field(default=20 ,descrition="Style strength of Ref Image (%)")
        guidance_scale: float = Field(default=5.0)
        seed_: int = Field(default=0)
        sa32_:float = Field(default=0.5)
        sa64_:float = Field(default=0.5)
        id_length_ : int = Field(default=2,descrition="Number of id images in total images")
        general_prompt : str = Field(...,descrition="Textual Description for Character")
        negative_prompt : str =  Field(default="naked, deformed, bad anatomy, disfigured, poorly drawn face, mutation, extra limb, ugly, disgusting, poorly drawn hands, missing limb, floating limbs, disconnected limbs, blurry, watermarks, oversaturated, distorted hands, amputation")
        prompt_array: str =  Field(...,descrition="Comic Description (each line corresponds to a frame)")
        G_height: int = Field(default=512)
        G_width: int = Field(default=512)
        comic_type : Literal['No typesetting (default)','Four Pannel','Classic Comic Style'] = Field(default='Classic Comic Style')
        font_choice :str = Field(default='Inkfree.ttf')
    """
    
    def __init__(self,endpoint_name,profile):
        endpoint_name = endpoint_name
        # boto_session= boto3.Session(profile_name=profile)
        boto_session= boto3.Session()
        self.s3_resource = boto_session.resource("s3")
        sagemaker_session = sagemaker.Session(boto_session = boto_session)
        bucket  = sagemaker_session.default_bucket()
        output_path  = "s3://{0}/{1}/asyncinvoke/out/".format(bucket, "story-diffusion")
        input_path :str = "s3://{0}/{1}/asyncinvoke/in/".format(bucket, "story-diffusion")
        
        predictor_ = Predictor(
            endpoint_name=endpoint_name,
            sagemaker_session=sagemaker_session,
            model_data_input_path=input_path,
            model_data_output_path=output_path,
        )
        predictor_.serializer = JSONSerializer()
        predictor_.deserializer = JSONDeserializer()
        self.config = WaiterConfig(
            max_attempts=100, delay=10  #  number of attempts  #  time in seconds to wait between attempts
        )
        self.predictor_async = AsyncPredictor(
                predictor_,
                name='story-diffusion'
        )
    
    # def get_async_result(self,prediction):
    #     images = []
    #     try:
    #         output_bucket, output_key = get_bucket_and_key(prediction.output_path)
    #         output_obj = self.s3_resource.Object(output_bucket, output_key)
    #         body = output_obj.get()["Body"].read().decode("utf-8")
            
    #         respobj = json.loads(body)
    #         print(respobj)
    #         for img in respobj['images_base64']:
    #             images.append(base64_to_image(img))
    #     except Exception as e:
    #         print(e)
    #     return images

    def generate_images(self,general_prompt:str,prompt_array:str,id_length:int=2, ref_imgs: List[Any]= [],comic_type:str='Classic Comic Style', style:str = 'Japanese Anime',sd_type:str="Unstable", height:int = 512, width :int = 768) -> list:
        data = { "general_prompt": general_prompt,
                        "prompt_array" : prompt_array,
                        "style" : style,
                        "G_height" : height,
                        "G_width" : width,
                        "comic_type" : comic_type,
                       "files":ref_imgs,
                        "id_length_":id_length,
                        "sd_type":sd_type,
                }
        if not ref_imgs:
            del data['files']
        # print(data)
        prediction = self.predictor_async.predict_async(data)
        print(f"Response output path: {prediction.output_path}")
        start = time.time()
        prediction.get_result(self.config)
        print(f"Time taken: {time.time() - start}s")
        
        output_bucket, output_key = get_bucket_and_key(prediction.output_path)
        output_obj = self.s3_resource.Object(output_bucket, output_key)
        body = output_obj.get()["Body"].read().decode("utf-8")
        
        respobj = json.loads(body)
        images = []
        for img in respobj['images_base64']:
            images.append(base64_to_image(img))
            
        # images =get_async_result(prediction)
        return images

- generate image from model in ec2 server 

In [ ]:
import requests
def generate_images_api(general_prompt:str,prompt_array:str,id_length:int=2, ref_imgs: List[Any]= [],comic_type:str='Classic Comic Style', style:str = 'Japanese Anime',sd_type:str="Unstable",height:int = 768, width :int = 768) -> list:
    data = { "general_prompt": general_prompt,
                    "prompt_array" : prompt_array,
                    "style" : style,
                    "G_height" : height,
                    "G_width" : width,
                    "comic_type" : comic_type,
                    "files":ref_imgs,
                    "id_length_":id_length,
                    "sd_type":sd_type,
            }
    if not ref_imgs:
        del data['files']
    payload = json.dumps(data)
    headers = {
        'Content-Type': 'application/json'
        }

    url = os.environ['api_endpoint']
    
    response = requests.request("POST", url, headers=headers, data=payload)
    images = []
    for img in response.json()['images_base64']:
        images.append(base64_to_image(img))
    return images

def generate_images_sdxl_api(prompt:str,height:int = 768, width :int = 768,style_preset=StyleEnum.Photographic.value) :
    data = { "prompt": prompt,
            "G_height":height,
            "G_width":width,
            "style_preset":style_preset}
    payload = json.dumps(data)
    headers = {
        'Content-Type': 'application/json'
        }

    url = os.environ['api_endpoint']
    
    response = requests.request("POST", url, headers=headers, data=payload)
    images = []
    for img in response.json()['images_base64']:
        images.append(base64_to_image(img))
    return images[0]

In [ ]:
import os
from io import BytesIO


def save_image_file(image, filename, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    image.save(os.path.join(folder, filename))
    print(f"image saved in {os.path.join(folder, filename)}")
    
def save_all_images(images,folder='./images'):
    img_fnames = []
    for i,img in enumerate(images):
        fname = str(i)+'.png'
        if img:
            save_image_file(img,fname,folder)
            img_fnames.append(os.path.join(folder, fname))
        else:
            img_fnames.append(None)
    return img_fnames

def save_all_images(images:list,file_names:list,folder:str ='./images'):
    img_fnames = []
    for img,fname in zip(images,file_names):
        fname =fname +'.png'
        if img:
            save_image_file(img,fname,folder)
            img_fnames.append(os.path.join(folder, fname))
        else:
            img_fnames.append(None)
    return img_fnames


def base64_to_image(base64_string):
    image_bytes = base64.b64decode(base64_string)
    image_buffer = BytesIO(image_bytes)
    image = Image.open(image_buffer)
    return image


def Image2base64(img_path):
    image = Image.open(img_path)
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    image_data = buffer.getvalue()
    base64_encoded_string = base64.b64encode(image_data).decode('utf-8')
    return base64_encoded_string

- test generation

In [ ]:
from IPython.display import display
# model_id='amazon.titan-image-generator-v1'

model_id='stability.stable-diffusion-xl-v1'
image_generator = ImageGenerator(model_id=model_id,profile='')
img = image_generator.generate_image(
  "A confident young woman in a stylish tailored suit with bold accessories, standing in a professional setting with an assertive posture and facial expression, photorealistic, sharp focus, studio lighting"
)
# img = generate_images_sdxl_api(prompt= "A confident young woman in a stylish tailored suit with bold accessories, standing in a professional setting with an assertive posture and facial expression, photorealistic, sharp focus, studio lighting")

display(img)

In [ ]:
endpoint = "story-diffusion-inference-api-2024-05-17-14-07-20-411"
sd_image_generator = StoryDiffusionGenerator(endpoint_name=endpoint,profile='')
images = sd_image_generator.generate_images(general_prompt = 'a girl',prompt_array="wake up in the bed\nhave breakfast",height=768,width=768)

In [ ]:
# images = generate_images_api(general_prompt = 'a girl',prompt_array="wake up in the bed\nhave breakfast",height=768,width=768,comic_type='No typesetting (default)')

In [ ]:
[display(img) for img in images]

In [ ]:
images

In [ ]:
from typing import List,Any
def get_final_state_env_var(steps:List[Any],node_name:str):
    answer = None
    for s in steps:
        for k in list(s.keys()):
            if k == node_name:
                answer = s[node_name]['env_var']
    return answer



from docx import Document
from docx.shared import Inches
from docx2pdf import convert
def save_image( image, folder='./images') -> str:
    """Save the image to a temporary file and return the file path."""
    image_path = f"temp_{hash(image.tobytes())}.png"
    filename = os.path.join(folder,image_path)
    image.save(filename)
    return filename


def save_as_docx(story, fname):
    document = Document()

    document.add_heading(story.page_title, 0)

    img_idx = 0
    for chapter in story.chapters:
        document.add_heading(chapter.chapter_title, level=1)
        for para in chapter.paragraphs:
            print(para)
            image = story.images[img_idx]
            img_idx += 1
            document.add_paragraph(para.content)

            # Add the image
            if image:
                document.add_picture(save_image(image), width=Inches(6))
            else:
                document.add_picture('placeholder.png', width=Inches(6))

    # Save the document
    document.save(fname)
    print(f'docx file saved as: {fname}')
    
    # Convert the Word document to PDF
    # pdf_path = f"{fname}.pdf"
    # convert(fname, pdf_path)
    # print(f'pdf file saved as: {pdf_path}')



def save_as_docx_few(characters,story, fname,suffix=''):
    document = Document()

    document.add_heading(story.page_title, 0)
    document.add_heading('Characters introduction', level=1)
    character = characters.main_character
    document.add_heading(f"{character.name}", level=2)
    document.add_paragraph(f"Role: {character.role}\nAffiliation: {character.affiliation}\nBackground: {character.background}")
    document.add_picture(f'./images/{character.name}{suffix}.png', width=Inches(4))
    for character in characters.supporting_character:
        document.add_heading(f"{character.name}", level=2)
        document.add_paragraph(f"Role: {character.role}\nAffiliation: {character.affiliation}\nBackground: {character.background}")
        document.add_picture(f'./images/{character.name}{suffix}.png', width=Inches(4))

    img_idx = 0
    for chapter in story.chapters:
        document.add_heading(chapter.chapter_title, level=1)
        # Add the image
        image = story.images[img_idx]
        img_idx += 1
        if image:
            document.add_picture(save_image(image), width=Inches(6))
        else:
            document.add_picture('placeholder.png', width=Inches(6))
        # Add paragraphs
        for para in chapter.paragraphs:
            print(para)
            document.add_paragraph(para.content)

    # Save the document
    document.save(fname)
    print(f'docx file saved as: {fname}')
    
    # Convert the Word document to PDF
    pdf_path = f"{fname}.pdf"
    try:
        convert(fname, pdf_path)
        print(f'pdf file saved as: {pdf_path}')
    except Exception as e:
        print(e)
        print('pdf file not saved')
    



### 1.4 Agent Roles settings

In [ ]:
company_setting = """You are woking in a cartoon studio, the best and creative cartoon studio in the world.\n"""


role_config = {
"cartoonist":  
      company_setting+"""You are a cartoonist.
Your task is to write an outline for a comics book about a user-provided topic. Be comprehensive and specific. And keep the outline as long as possible.
You can refine your story if there is suggestion provided by other roles in your studio 
      """,
      
"screenwriter":  
      company_setting+"""You are a Screenwriter.
Your task is to create a main character and a diverse and distinct group of supporting characters for a new story, based on the provided topic and outline.
For each supporting character, please provide the following:
1. A unique name and role in the story (e.g. sidekick, mentor, rival, etc.)
2. A brief description of their perspective, affiliation, or background related to the story's themes
3. An explanation of what aspects of the story they will focus on or influence
Additionally, think step-by-step about how to make this group of characters distinct and complementary to create an engaging, multifaceted narrative
""",

"editor": 
      company_setting+"""You are a comics book editor, you can proofread and provide suggestions on improving the content of Plot design of the book.
Here is outline of a comics book: 
<outline>
{outline}
</outline>
You are now required to proofread and provide suggestions on specific chapter based on the outline, with the following aspects:
<aspects>
  1. it should consider the context of other chapters in the outline to continue writing your specific chapter
  2. it should consider contradictory plots with other chapter, for example a character who has gone forever in other chapter appearing again in the chapter you are writing
  3. it should consider topics such as pornography, racial discrimination, and toxic content
  4. it should be compelling and attract young people
  4. Any other suggestions which you think can improve the content
</aspects>
    """,
"art designer":
"""You are an art designer. 
Your task is to generate creative art design ideas and use a Stable Diffusion model to generate high-quality portrait images for the characters in a book,
You need to create prompt for Stable Diffusion model with the following instructions:
<instructions>
1. Consider adding modifiers like aspect ratios, image quality settings, or post-processing instructions to refine the output.
2. Avoid topics such as pornography, racial discrimination, and toxic words.
3. Be concise and less then 30 words.
4. the prompt should always be English
5. do not output the character's name, use more general identity instead, such as a young man, an old women, a teenager boy etc.
</instructions>
Here is example:
<example>
Prompt: A highly detailed, photorealistic portrait of a young woman with long, curly red hair, fair skin, and piercing green eyes, standing in front of a window overlooking a lush forest, soft natural lighting, 4k, artstation, unreal engine
</example>
""",

"story illustrator":
      company_setting+ """You are a story illustrator. 
You first identify the referential relationship in the description and replace the pronouns with the names of the characters, and output your the result in <intermediate_step> tag.
Then You need to break down the description in <intermediate_step> into several short sentences with subject-predicate-object relationships, maintaining a certain logical order between the short sentences.
put your final answer in xml tag <answer>
You need to follow instructions:
<instructions>
1. Use descriptive keywords or short sentences to convey the desired content, style, action.
2. Avoid topics such as pornography, racial discrimination, and toxic words.
3. Each sentence should be concise and less then 10 words.
4. the output should always be English.
5. each sentence in one line
6. no more then 8 sentences in total
7. If thera are characters name exists in the short sentence,  add brackets to enclose the name. for example, [Bob] invited [Alice] to join him on an adventure
8. If there are no characters in the short sentence, add a [NC] symbol at the beginning of the sentence.For example, to generate a scene of falling leaves without any character, write: [NC] The leaves are falling.
</instructions>
Here is example:
<example>
input:Bob was at home, reading the newspaper. An article caught his eye – it mentioned a treasure house hidden deep in the forest. Intrigued, Bob decided to embark on an adventure to find this elusive treasure.
On the road, near the forest, Bob invited his friend Alice to join him on this exciting quest. Alice, always up for an adventure, eagerly agreed.
They drove to the forest, their car winding through the narrow roads, as they ventured deeper into the unknown. As night fell, the forest took on an eerie atmosphere. Suddenly, a tiger appeared, its piercing eyes glowing in the darkness.
Bob and Alice were terrified, their mouths agape in fear. Without hesitating, they ran as fast as they could, their hearts pounding in their chests.
output:
<answer>
[Bob] at home, read new paper #at home, The newspaper says there is a treasure house in the forest
[Bob] on the road, near the forest
[Alice] is make a call at home # [Bob] invited [Alice] to join him on an adventure.
[NC] The car on the road, near the forest #They drives to the forest in search of treasure.
[NC]A tiger appeared in the forest, at night 
[Bob] very frightened, open mouth, in the forest, at night
[Alice] very frightened, open mouth, in the forest, at night
[Bob]  running very fast, in the forest, at night
</answer>
</example>
""",


"translator":
      company_setting+"""You are the world's most professional translator, proficient in professional translation between {source_lang} and {dist_lang}. 
We are testing your output and translation capabilities. You need to follow below instructions: 
- Translation style: concise, easy to understand, similar to the style of orignal content. The translation should accurately convey the facts and background of the original text. 
- Avoid quotation mark within a quotation mark, if encountering a quotation mark within a quotation mark, it needs to be single quotation mark instead
Do not try to explain the content to be translated, your task is only to translate. """,

"summarizer":
"""You are an expert paragraph summarizer. Your task is to summarize a given paragraph into a concise, short sentence while capturing the main idea or key point of the paragraph.
Skip the premble, output the summary directly.
""",

}

- structured output prompt

In [ ]:
fc_desc = """
You will ALWAYS follow the below guidelines when you are answering a question:
<guidelines>
- Think through the user's question, extract all data from the question and the previous conversations before creating a plan.
- Always output your thoughts within <thinking></thinking> xml tags before you respond to the user. 
- Your response must be follow the pydantic schema as:
<schema>
{schema}
</shema>
- output your answer in json format, enclosed in tag <answer>, so that the user can use pydantic basemodel.parse_obj() to parse the json string into an object which defined as:
 <schema>
 {schema}
 </shema>
- Avoid quotation mark within a quotation mark, if encountering a quotation mark within a quotation mark, it needs to be single quotation mark instead
- if the content has quotation mark, please change to single quotation mark instead
</guidelines>
"""

## 2. Workflow_1: Generate Outline Workflow

### 2.1 generate initial outline

For many topics, your LLM may have an initial idea of the important and related topics. We can generate an initial
outline to be refined after our research. Below, we will use our "fast" llm to generate the outline.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage

class Chapter(BaseModel):
    chapter_title: str = Field(..., title="Title of the chapter")
    description: str = Field(..., title="Summary description of the chapter")

    @property
    def as_str(self) -> str:
        # chapter_content = "\n".join(self.paragrahs)
        return f"## {self.chapter_title}\n\n{self.description}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the comics book")
    chapters: List[Chapter] = Field(
        default_factory=list,
        min_items=8,
        # max_items=20,
        title="Titles and descriptions for each chapter of the comics book.",
    )

    @property
    def as_str(self) -> str:
        chapter = "\n\n".join(chapter.as_str for chapter in self.chapters)
        return f"# {self.page_title}\n\n{chapter}".strip()


- prompt template 

In [ ]:
direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            role_config["cartoonist"]+ fc_desc,
        ),
        # ("user",  "Write an outline for a comics book about a user-provided topic. Be comprehensive and specific. And keep the outline as long as possible Here is {topic}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [ ]:
topic = "A little boy who lived in a mountain village obtained a magic book in the wild. He became self-taught and became the most powerful magician in the world, helping the world resist alien invasions. "
# topic = "续写一位霸道女总裁的故事。她从外交学院毕业之后，经过多年奋斗，终于进入了一家知名企业当副总裁，然而她对下属员工非常刻薄，她在个人抖音账号上发布了多条视频，包括“员工闹分手提离职我秒批”、“300封举报信撒满工位”等话题，这些视频内容引发了公众的热议和争议，终于被公司辞退。被公司辞退前她自己注册了一家公司。"
generate_outline_direct = direct_gen_outline_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) | RunnableLambda(dict_to_obj).bind(target=Outline)

- test for generation
- If you encounter JSONDecodeError, please rerun again

In [ ]:
outline = generate_outline_direct.invoke({"messages": [HumanMessage(content=f"Here is the topic:{topic}")],"schema":Outline.schema_json() })

### 2.2 Create characters

In [ ]:
class Persona(BaseModel):
    affiliation: str = Field(
        description="Primary affiliation of the character.",
    )
    name: str = Field(
        description="only first name of character, need to match with '^[a-zA-Z0-9_-]{1,64}$'",
    )
    role: str = Field(
        description="Role of the character in the story.",
    )
    background: str = Field(
        description="background of the person in the story.",
    )
    description: str = Field(
        description="description of the character’s personality, hobbies, etc.",
    )
    figure: str = Field(
        description="figure representing such as a boy,a girl,a man, a women, a young woman,an old man or etc",
    )
    appearance: str = Field(
        description="appearance, attire of the character in the story.",
    )

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nBackground: {self.background}\nDescription: {self.description}\nFigure:{self.figure}\nAppearance: {self.appearance}\n"
    
class Character(BaseModel):
    main_character: Persona = Field(
        description="the main character in the story.",
    )
    
    supporting_character: List[Persona] = Field(
        description="Comprehensive list of supporting characters in the story.",
        min_items=2,
        max_items=4,
    )
    @property
    def as_str(self) -> str:
        return "\n".join([e.persona for e in self.supporting_character])+'\n'+self.main_character.persona

In [ ]:
gen_character_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
           role_config["screenwriter"]+ fc_desc,
        ),
         MessagesPlaceholder(variable_name="messages")
    ]
)
gen_characters_chain = gen_character_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=Character)

- test for generation

In [ ]:
request = HumanMessage(content=f"Here is the outline:{outline.as_str}")
characters = gen_characters_chain.invoke({"messages": [request],"schema":Character.schema_json()})

In [ ]:
print(characters.as_str)

### 2.3 Add charaters to refine outline

-  test for refine generation

In [ ]:
outline = generate_outline_direct.invoke({"messages": [HumanMessage(content=f"Here is the topic:{topic}"),
                                                               AIMessage(content=f"Here is the outline draft:{outline.json()}, "),
                                                               HumanMessage(content=f"Here is the characters description:{characters.json()}.\n Your task is to rewrite the outline draft for a comics book based on the outline draft. Please incorporate all the characters in the story, and keep the outline Be comprehensive and specific ")],
                                          "schema":Outline.schema_json() })

In [ ]:
print(outline.as_str)

### 2.4 build outline workflow graph

In [ ]:
from langgraph.graph import StateGraph, END

In [ ]:
import operator
from typing import Annotated, Sequence,Dict,Optional,Any,TypedDict

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    env_var: Optional[Annotated[Dict[str, Any], operator.ior]]

In [ ]:
def convert_message_name(message:BaseMessage):
    if isinstance(message, AIMessage) and message.name:
        return AIMessage(content=f"{message.name} : {message.content}")
    elif isinstance(message, HumanMessage) and message.name:
        return HumanMessage(content=f"{message.name} : {message.content}")
    else:
        return message

##merge the continouse roles, and change sequences 
def reconstruct_to_claude_messages(messages):
    rec_messages = []
    for message in messages:
        message = convert_message_name(message)
        if rec_messages:
            if isinstance(rec_messages[0], AIMessage):
                rec_messages[0] = HumanMessage(content=rec_messages[0].content)
            last_msg = rec_messages[-1]
            last_role = 'assistant' if isinstance(last_msg,AIMessage) else 'user'
            current_role = 'assistant' if isinstance(message,AIMessage) else 'user'
            if last_role == current_role:
                last_msg_content = last_msg.content[-1]['text'] if isinstance(last_msg.content,list) else last_msg.content
                current_msg_content = message.content[-1]['text'] if isinstance(message.content, list) else message.content
                new_content = last_msg_content +"\n\n" + current_msg_content
                rec_messages[-1] = HumanMessage(content=new_content) if last_role == 'user' else AIMessage(content=new_content)
            else:
                rec_messages.append(message)
        else:
            rec_messages.append(message)
    return rec_messages

def swap_roles(messages, name: str):
    converted = []
    for message in messages:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    return  converted

In [ ]:

from langchain_core.pydantic_v1 import ValidationError

async def retry_call(chain,args: Dict[str,Any],times:int=5):
    try:
        content = await chain.ainvoke(args)
        return content
    except JSONDecodeError as e:
        if times:
            print(f'JSONDecodeError, retry again [{times}]')
            return await retry_call(chain,args,times=times-1)
        else:
            raise(JSONDecodeError(msg='JSONDecodeError'))
    except ValidationError as e:
        return "end"

In [ ]:
async def generate_outline(state:AgentState):
    """
    Generate outline

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with outline
    """
    print("---generate_outline---")
    env_var =  state.get("env_var")
    assert not env_var is None
    
    generate_outline_chain = direct_gen_outline_prompt | llm | CustJsonOuputParser(verbose=True) | RunnableLambda(dict_to_obj).bind(target=Outline)
    name = "cartoonist"
    messages = swap_roles(state['messages'],name)
    messages = reconstruct_to_claude_messages(messages)
    outline = await retry_call(generate_outline_chain,{"messages": messages,"schema":Outline.schema_json() })
    response = AIMessage(content=f"Here is the outline: \n{outline.json()}",name=name)
    return {"messages":[response],"env_var":{**env_var,"outline":outline}}

async def generate_characters(state:AgentState):
    """
    Generate characters

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with outline
    """
    print("---generate_characters---")
    env_var =  state.get("env_var")
    assert not env_var is None
    
    name = 'screenwriter'
    messages = swap_roles(state['messages'],name)
    messages = reconstruct_to_claude_messages(state['messages'])
    gen_characters_chain = gen_character_prompt | llm | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=Character)
    characters = await retry_call(gen_characters_chain,{"messages": messages,"schema":Character.schema_json() })
    response = AIMessage(content=f"Here is the characters description:\n{characters.json()}.\n Your task is to rewrite the outline draft for a story based on the outline draft. Please incorporate all the characters in the story, and keep the outline be comprehensive and specific ",name=name)

    return {"messages":[response],"env_var":{**env_var,"characters":characters}}

In [ ]:
MAX_TURNS = 2
def should_repeat_outline(state: AgentState):
    messages = state['messages']
    # print(messages)
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage)]
    )
    print('messages:',num_responses)
    if num_responses > MAX_TURNS:
        return 'end'
    else:
        return 'generate_characters'

In [ ]:
outline_graph = StateGraph(AgentState)
outline_graph.add_node("generate_outline",generate_outline)
outline_graph.add_node("generate_characters",generate_characters)
outline_graph.set_entry_point("generate_outline")
outline_graph.add_edge("generate_characters", "generate_outline")
outline_graph.add_conditional_edges("generate_outline",
                                   should_repeat_outline,
                                   {
                                       'end':END,
                                       'generate_characters':'generate_characters'
                                   }
                                   )
outline_workflow = outline_graph.compile()

In [ ]:
from IPython.display import Image

Image(outline_workflow.get_graph().draw_png())



* test generation
- Sometimes there is exception JSONDecodeError, you should rerun the flow

In [ ]:
steps = []
init_state = {
    "env_var":{"topic":topic},
    "messages":[HumanMessage(content=f"Here is the topic:{topic}")]
}
async for event in outline_workflow.astream(init_state):
    print(event)
    # print(list(event.values())[0]['env'])
    print("---")
    steps.append(event)

In [ ]:
env_var =  steps[-1]['generate_outline']['env_var']
outline = env_var['outline']
characters = env_var['characters']

In [ ]:
print(outline.as_str)

In [ ]:
print(characters.as_str)

## 3. Workflow_2 - write story sections in paraller

### 3.1 define the output structure

In [ ]:
class Paragragh(BaseModel):
    content: str = Field(..., title="Content of the Paragrah")

class DetailChapter(BaseModel):
    chapter_title: str = Field(..., title="Title of the chapter")
    paragraphs: List[Paragragh] = Field(..., title="List of paragraghs of the chapter")
    
    @property
    def as_str(self) -> str:
        chapter_content = "\n".join([p.content for p in self.paragraphs])
        return f"## {self.chapter_title}\n\n{chapter_content}".strip()

In [ ]:
write_chapter_prompt = ChatPromptTemplate.from_messages( 
 [    
   (
        "system",
        role_config["cartoonist"]  + 
         """Here is the outline of the story: 
      <outline>
      {outline}
      </outline>
      Here is the characters of the story:
      <characters>
      {characters}
      </characters>
      You are now required to write stories for specific chapter based on the outline and characters, with the following requirements:
      <requirements>
        1. You need to consider the context of other chapters in the outline to continue writing your specific chapter
        2. You can only use the characters to write the story, don't create any other characters beyond the provided characters.
        3. Avoid contradictory plots with other chapter, for example a character who has gone forever in other chapter appearing again in the chapter you are writing
        4. Avoid topics such as pornography, racial discrimination, and toxic content
      </requirements>""" + fc_desc
        ),
    MessagesPlaceholder(variable_name="messages", optional=True),  
 ]
)

In [ ]:
outline.chapters[0].json()

- test generation

In [ ]:
gen_chapter_chain = write_chapter_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=DetailChapter)

In [ ]:
chapter_obj = await gen_chapter_chain.ainvoke({"outline":outline,"messages":[HumanMessage(content = outline.chapters[0].json())],"characters":characters.as_str,"schema":DetailChapter.schema_json()})

In [ ]:
print(chapter_obj.as_str)

### 3.2 Editor to refine the chatper

In [ ]:
class EditorSuggestion(BaseModel):
  suggestions: List[str] = Field(
      description="suggestion list",
      max_items=10,
  )
  @property
  def as_str(self) -> str:
      return "\n".join([f"{i+1}.{e}" for i,e in enumerate(self.suggestions)])

refine_chapter_prompt = ChatPromptTemplate.from_messages( 
 [    
   (
        "system",
        role_config["editor"],
        ),
    MessagesPlaceholder(variable_name="messages", optional=True),  
 ])

In [ ]:
refine_chapter_chain = refine_chapter_prompt | llm_sonnet | StrOutputParser()

- test generation

In [ ]:
refine_suggestions = refine_chapter_chain.invoke({"outline":outline.json(),"messages":[HumanMessage(content=chapter_obj.json())]})

In [ ]:
print(refine_suggestions)

### 3.3 Build workflow graph

In [ ]:


async def write_chapter(state: AgentState):
    messages = state["messages"]
    name = 'cartoonist'
    messages = swap_roles(messages,name)
    # print(messages)
    env_var = state['env_var']
    outline = env_var['outline']
    characters = env_var['characters']
    assert not outline == None
    messages = reconstruct_to_claude_messages(messages)
    assert not outline == None
    gen_chapter_chain = write_chapter_prompt | llm | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=DetailChapter)
    chapter_obj= await retry_call(gen_chapter_chain,{"outline":outline.json(),"messages":messages,"characters":characters.as_str,"schema":DetailChapter.schema_json()})
    
    
    ## sometimes the agent would end the converastion, so it would not follow the structured output. we should end converastion then
    if isinstance(chapter_obj,DetailChapter):
        formatted_message = AIMessage(name=name, content=chapter_obj.json())
        return {"messages":[formatted_message],"env_var":{**env_var,"chapter":chapter_obj}}
    else:
        return {"messages":[AIMessage(name=name, content="Let's end the coversation")],"env_var":{**env_var}}
    

In [ ]:
async def refine_chapter(state: AgentState):
    messages = state["messages"]
    name = "editor"
    messages = swap_roles(messages,name)
    # print(messages)
    env_var = state['env_var']
    outline = env_var['outline']
    assert not outline == None
    messages = reconstruct_to_claude_messages(messages)
    refine_chapter_chain = refine_chapter_prompt | llm | StrOutputParser()
    # suggestion = await refine_chapter_chain.ainvoke({"outline":outline,"messages":messages})
    suggestion = await retry_call(refine_chapter_chain,{"outline":outline.json(),"messages":messages})
    formatted_message = AIMessage(name=name, content=suggestion)
    return {"messages":[formatted_message],"env_var":{**env_var}}
    

In [ ]:
MAX_TURNS = 2
def should_repeat_write(state: AgentState):
    messages = state['messages']
    last_msg = messages[-1].content
    # print(messages)
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage)]
    )
    print(f'num_responses:{num_responses}')
    if num_responses > MAX_TURNS or last_msg.startswith("Let's end the coversation"):
        return 'end'
    else:
        return 'refine_chapter'

In [ ]:
write_graph = StateGraph(AgentState)

write_graph.add_node("write_chapter",write_chapter)
write_graph.add_node("refine_chapter",refine_chapter)
write_graph.set_entry_point("write_chapter")

write_graph.add_edge("refine_chapter","write_chapter")
write_graph.add_conditional_edges("write_chapter",
                                  should_repeat_write,
                                  {
                                      "end":END,
                                      "refine_chapter":"refine_chapter"
                                  })
write_workflow = write_graph.compile()


In [ ]:
Image(write_workflow.get_graph().draw_png())

- let's test, write a single chapter

In [ ]:
steps = []
init_state = {
    "env_var":{"outline":outline,"characters":characters},
    "messages":[HumanMessage(content=f"Here is the origin content: {outline.chapters[0].json()}",name='editor')]
}
async for event in write_workflow.astream(input=init_state):
    steps.append(event)
    for key, value in event.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

In [ ]:
steps

In [ ]:
print(get_final_state_env_var(steps,'write_chapter')['chapter'])

- let's use batch function to write each chapter

In [ ]:
chaps = [ chatper for chatper in  outline.chapters]
chaps


In [ ]:
async def write_all_chapters(chaps,outline,characters):
    init_state = {
    "env_var":{"outline":outline,"characters":characters,"chapter":None},
    "messages":[HumanMessage(content=f"Here is the origin content:\n {chaps.json()}",name='editor')]
    }       
    steps = []
    async for event in write_workflow.astream(input=init_state):
        steps.append(event)
        for key, value in event.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value)
        print("\n---\n")
    ret = get_final_state_env_var(steps,'write_chapter')['chapter']
    # trans =  get_final_state_env_var(steps,'translate_chapter')['translation']
    return (ret,None)

In [ ]:
write_all = RunnableLambda(write_all_chapters).bind(outline=outline,characters=characters)
all_steps =  await write_all.abatch(chaps)

In [ ]:
len(all_steps)

In [ ]:
# all_steps

### merge all Subchapters together 

In [ ]:


class Story(BaseModel):
    page_title: str = Field(..., title="Title of the comics book")
    chapters: List[DetailChapter] = Field(
        default_factory=list,
        title="Titles and descriptions for each chapter of the comics book.",
    )
    images: Optional[List[Any]] = Field(default=[], title="List of illustration for each chapter")
    
    @property
    def as_str(self) -> str:
        chapter = "\n\n".join(chapter.as_str for chapter in self.chapters)
        return f"# {self.page_title}\n\n{chapter}".strip()

In [ ]:
print(len(outline.as_str))

In [ ]:
story = Story(page_title=outline.page_title,chapters=[s[0] for s in all_steps])
print(len(story.as_str))
print(story.as_str)

## 4. Workflow_4 - Art design -generate Images

### 4.1 setup prompt optimzer for SDXL and StoryDiffusion Model

- Define structured output

In [ ]:
class ImagePrompt(BaseModel):
    prompt: str = Field(
        description="an optimized prompt for the Stable Diffusion model based on the given instructions and guidelines, it should always be English",
    )

### 4.2 Firstly generate character identity images using sdxl

In [ ]:
gen_identity_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            role_config['art designer']+fc_desc,
        ),
        MessagesPlaceholder(variable_name="messages",optional=True),  
    ]
)

In [ ]:
gen_identity_chain = gen_identity_prompt | llm_sonnet | CustJsonOuputParser(verbose=False) |RunnableLambda(dict_to_obj).bind(target=ImagePrompt)

- test for generation

In [ ]:
sd_prompt = gen_identity_chain.invoke({'messages':[HumanMessage(content=characters.main_character.appearance)],'schema':ImagePrompt.schema_json()})
sd_prompt

In [ ]:
# image_generator = ImageGenerator(profile='')

- generate all identity images and saved to later story diffusion

In [ ]:

def generate_all_identities(characters,gen_identity_chain):
    batch_inputs  = [{'messages':[HumanMessage(content=characters.main_character.appearance)],'schema':ImagePrompt.schema_json()}]
    character_names = [characters.main_character.name]
    for ch in characters.supporting_character:
        batch_inputs += [{'messages':[HumanMessage(content=ch.appearance)],'schema':ImagePrompt.schema_json()}]
        character_names += [ch.name]
    
    sd_prompts = gen_identity_chain.batch(batch_inputs)
    
    image_generator = ImageGenerator(profile='')
    images = [image_generator.generate_image(prompt=sd_prompt.prompt) for sd_prompt in sd_prompts]
    
    # save images in folder
    save_all_images(images,character_names)
    return images,sd_prompts
    
    

In [ ]:
_,sd_prompts = generate_all_identities(characters,gen_identity_chain)

### 4.3 Generate storylines for story diffusion model

In [ ]:
# class StoryLines(BaseModel):
#     sentences : List[str] = Field (description="""break down a description into several short sentences with subject verb object relationships, maintaining a certain logical order between the sentences. 
#                                    If there are no characters in the short sentence, add a [NC] symbol at the beginning of the sentence.
#                                    For example, to generate a scene of falling leaves without any character, write: [NC] The leaves are falling.""", 
#                                    max_items=8)

In [ ]:
gen_image_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            role_config['story illustrator']
        ),
        MessagesPlaceholder(variable_name="messages",optional=True),  
    ]
)

- This chain has to be Sonnet

In [ ]:
# gen_image_prompt_chain =  gen_image_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=StoryLines)
gen_image_prompt_chain =  gen_image_prompt | llm_sonnet | TextOuputParser(verbose=False)

In [ ]:
request = HumanMessage(content='\n'.join([ p.content for p in story.chapters[0].paragraphs]))

In [ ]:
# request = HumanMessage(content=story.chapters[0].paragraphs[2].content)
request = HumanMessage(content='\n'.join([ p.content for p in story.chapters[0].paragraphs]))
gen_image_prompt_chain.invoke({"messages":[request]})

- generate story lines for story diffusion in batch

In [ ]:
# create image by each paragraph cost very long time, so we only create by chapter instead
# messages = [{"messages":[HumanMessage(content=f"Here is the description:\n{para.content}")]} for chapter in story.chapters for para in chapter.paragraphs] 
messages = []
for chapter in story.chapters:
    messages += [{"messages":[HumanMessage(content='\n'.join([ p.content for p in chapter.paragraphs]))]}] 
# messages

In [ ]:
all_story_lines = gen_image_prompt_chain.batch(messages)

In [ ]:
len(all_story_lines)

In [ ]:
# pattern = r"\[(.*?)\]"
# story_lines_split = []
# for p in all_story_lines:
#     match = re.findall(pattern, p)
#     match_set = set(match)
#     match_set.discard('NC')
#     if len(list(match_set)) <= 2:
#         story_lines_split.append(p)
#     else:
#         # print('------------')
#         # print(p)
#         prompt_arry = p.splitlines()
#         counter = 0
#         unique_p = {} 
#         new_prompt_array = []
#         for prompt in prompt_arry:
#             # print(prompt)
#             if prompt.startswith('[NC]'):
#                 new_prompt_array.append(prompt)
#             for name in list(match_set):
#                 if f"[{name}]" in prompt:
#                     if prompt not in unique_p:
#                         new_prompt_array.append(prompt)
#                         unique_p[prompt] = 1
#                         counter += 1
#                 if counter > 2:
#                     story_lines_split.append(new_prompt_array)
#                     new_prompt_array = []
#                     unique_p = {}
#                     counter = 0
#                     break

# [print('---:',p) for p in story_lines_split]         
                
            

- each story line will send to storydiffusion model to create a comic, count the characters in each line and add crespondant ref images

In [ ]:
from PIL import Image
import re
def count_character_names(character_names,line):
    name_counter = {}
    for name in character_names:
        if f"[{name}]" in line:
            if name in name_counter:
                name_counter[name] += 1
            else:
                name_counter[name] = 1
    return name_counter if name_counter else {'[NC]':1}


#https://github.com/HVision-NKU/StoryDiffusion/blob/main/utils/gradio_utils.py
# convert character list to dict
def character_to_dict(general_prompt):
    character_dict = {}    
    generate_prompt_arr = general_prompt.splitlines()
    character_index_dict = {}
    invert_character_index_dict = {}
    character_list = []
    for ind,string in enumerate(generate_prompt_arr):
        # 分割字符串寻找key和value
        start = string.find('[')
        end = string.find(']')
        if start != -1 and end != -1:
            key = string[start:end+1]
            value = string[end+1:]
            if "#" in value:
                value =  value.rpartition('#')[0] 
            if key in character_dict:
                raise Exception("duplicate character descirption: " + key)
            character_dict[key] = value
            character_list.append(key)

        
    return character_dict 

def calc_id_length_prompt(general_prompt,prompts):
    character_dict = character_to_dict(general_prompt)
    replace_prompts = []
    character_index_dict = {}
    invert_character_index_dict = {}
    for ind,prompt in enumerate(prompts):
        for key in character_dict.keys():
            if key in prompt:
                if key not in character_index_dict:
                    character_index_dict[key] = []
                character_index_dict[key].append(ind)
                if ind not in invert_character_index_dict:
                    invert_character_index_dict[ind] = []
                invert_character_index_dict[ind].append(key)
        cur_prompt = prompt
        if ind in invert_character_index_dict:
            for key in invert_character_index_dict[ind]:
                cur_prompt = cur_prompt.replace(key,character_dict[key])
        replace_prompts.append(cur_prompt)
    # print(replace_prompts)
    ref_index_dict = {}
    ref_totals = []
    # print(character_index_dict)
    id_length = 999
    for character_key in character_index_dict.keys():
        if character_key not in character_index_dict:
            raise Exception("{} not have prompt description, please remove it".format(character_key))
        index_list = character_index_dict[character_key]
        # print(invert_character_index_dict)
        index_list = [index for index in index_list if len(invert_character_index_dict[index]) == 1]
        # print(f'{character_key}:',index_list)
        id_length = len(index_list) if len(index_list) < id_length else id_length
        # if len(index_list) < id_length:
        #     raise Exception(f"{character_key} not have enough prompt description, need no less than {id_length}, but you give {len(index_list)}")
        ref_index_dict[character_key] = index_list[:id_length]
        ref_totals = ref_totals + index_list[:id_length]
    return id_length
    
def generate_img_dicts(characters):
    character_names = [characters.main_character.name]
    name_figure_map = {characters.main_character.name:characters.main_character.figure}
    
    # add supporting characters
    for ch in characters.supporting_character:
        character_names += [ch.name]
        name_figure_map[ch.name] = ch.figure
        

    # print('name_figure_map:',name_figure_map)
    imgs = {}
    for key in list(name_figure_map.keys()):
        if key not in imgs :
            imgs[key] = Image2base64(f'./images/{key}.png')
    return imgs

    
def prepare_storyd_prompts(story_lines,characters,img_dicts):
    """
        prepare prompts for story diffusion model. 
        Use character's portrait as reference images to keep the consistance
    """
    character_names = [characters.main_character.name]
    name_figure_map = {characters.main_character.name:characters.main_character.figure}
    
    # add supporting characters
    for ch in characters.supporting_character:
        character_names += [ch.name]
        name_figure_map[ch.name] = ch.figure
        
    # count character names in each line
    name_counters = []
    for line in story_lines:
        name_counters.append(count_character_names(character_names,line))
    # print('name_counters:',name_counters)
    # generate prompt for each line
    args = []
    for name_counter,line in zip(name_counters,story_lines):
        # id_length = len(list(name_counter.keys()))
        ref_imgs = []
        figures = []
        for key in list(name_counter.keys()):
            if key != '[NC]':
                ref_imgs.append(img_dicts[key])
                figures.append(f"[{key}] {name_figure_map[key]} img")
            else:
                figures.append(f"[NC]")
                
        prompt_array = line.split("\n")

        # the model cannot generate one image with more than 2 character identities
        prompt_array_new = []
        pattern = r"\[(.*?)\]"
        for text in prompt_array:
            new_prompt = text
            match = re.findall(pattern, text)
            # check if the identity is in the first two of character dict
            if match and match[0] != 'NC' and match[0] not in list(name_counter.keys())[:2]:
                new_prompt = new_prompt.replace(f"[{match[0]}]",name_figure_map[key],1) 
                print(match[0], name_figure_map.keys(), name_counter.keys(),text,new_prompt)

            match = re.findall(pattern, new_prompt)
            if match and len(match) > 1:
                for k in match[1:]:
                    if k != 'NC': 
                        new_prompt = new_prompt.replace(f"[{k}]",name_figure_map[k]) 

            new_prompt = new_prompt if new_prompt.startswith('[NC]') else new_prompt

            match = re.findall(pattern, new_prompt)
            if not match:# if there is no bracket in the prompt line, it has to be add [NC]
                new_prompt = '[NC]' + new_prompt 
            
            prompt_array_new.append(new_prompt +'#' + text )# the text after # becomes caption 

        # calc id length
        general_prompt = '\n'.join(figures[:2]) #can only accept the first 2 figures currently, need to update in future              

        id_length = 2
        # add extral prompt for identity in case have enough prompt description for identity
        prompt_array_new = [f.replace(' img','') for f in figures[:id_length]] + prompt_array_new

        #re calc again after the prompt changed
        id_length = calc_id_length_prompt(general_prompt, prompt_array_new)
        id_length = 2 if id_length > 2 else id_length
        # print(prompt_array_new)    
        
        # now the model can only support max 2 ref images in general prompt
        # args.append({'prompt_array':prompt_array_new,'id_length':id_length,'ref_imgs':ref_imgs[:2],'general_prompt':general_prompt})
        yield({'prompt_array':prompt_array_new,'id_length':id_length,'ref_imgs':ref_imgs[:2],'general_prompt':general_prompt})
    # return args

In [ ]:
img_dicts = generate_img_dicts(characters)

In [ ]:
storyd_prompts = [r for r in prepare_storyd_prompts(all_story_lines,characters,img_dicts)]

In [ ]:
[ p['id_length'] for p in storyd_prompts]

In [ ]:
[ p['prompt_array'] for p in storyd_prompts]

In [ ]:
all_story_lines[2]

- use story diffusion model to refine the style of character images

### 4.2 Using StoryDiffusion to generate images

In [ ]:
endpoint = 'story-diffusion-inference-api-2024-05-17-14-07-20-411'
stod_image_generator = StoryDiffusionGenerator(endpoint_name=endpoint,profile='')


In [ ]:
# storyd_prompts[6]['prompt_array']

In [ ]:
images_list = []

- this step is very time consuming, while it takes 60s per line

In [ ]:
start_index = 0
# sytle = "Comic book"
sytle = "Japanese Anime"
comic_type = "Four Pannel"
sd_type = "Unstable"

for i,p in enumerate(storyd_prompts[start_index:]):
    print(f"line:{i},{p['prompt_array']}")  
    images = stod_image_generator.generate_images(general_prompt = p['general_prompt'],
                                                    style=sytle,
                                                    comic_type = comic_type,
                                                    prompt_array='\n'.join(p['prompt_array']),
                                                    id_length= p['id_length'],
                                                     sd_type = sd_type,
                                                    ref_imgs=p['ref_imgs'],height=1024,width=1024)
    # images = generate_images_api(
    #     general_prompt = p['general_prompt'],
    #                                                 style=sytle,
    #                                                 comic_type = comic_type,
    #                                                 sd_type = sd_type,
    #                                                 prompt_array='\n'.join(p['prompt_array']),
    #                                                 id_length= p['id_length'],
    #                                                 ref_imgs=p['ref_imgs'],height=768,width=768
    # )
    images_list.append(images)

In [ ]:
len(images_list)
# images_list

In [ ]:
images = [img for imgs in images_list for img in imgs[::-1][-3:]] ##reverse the output order
images

In [ ]:
[display(img) for img in images]

In [ ]:
story.images = images

In [ ]:
# save_as_docx_few(characters,story,f'{story.page_title}_few_images_2.docx')

### use story diffusion model to refine the style of character images

In [ ]:
def refine_character_images(characters):
    comic_type = "Four Pannel"
    character_names = [characters.main_character.name]
    name_figure_map = {characters.main_character.name:characters.main_character.figure}
    
    # add supporting characters
    for ch in characters.supporting_character:
        character_names += [ch.name]
        name_figure_map[ch.name] = ch.figure
    character_images = []
    for p,name in zip(sd_prompts,character_names):
        
        prompt_array = f"[{name}] {p.prompt}"
        general_prompt = f"[{name}] "+name_figure_map[name]+ ' img'
        print(f"general_prompt:{general_prompt}\nprompt_array:{prompt_array}")
        ref_imgs = [img_dicts[name]]
        images = stod_image_generator.generate_images(general_prompt = general_prompt,
                                                        style=sytle,
                                                        comic_type = comic_type,
                                                        prompt_array=prompt_array,
                                                        id_length=1,
                                                        ref_imgs=ref_imgs,
                                                        height=768,
                                                        width=768)
        # images = generate_images_api(general_prompt = general_prompt,
        #                                                 style=sytle,
        #                                                 comic_type = comic_type,
        #                                                 prompt_array=prompt_array,
        #                                                 id_length=1,
        #                                                 ref_imgs=ref_imgs,
        #                                                 height=768,
        #                                                 width=768)
        character_images.append(images)
    save_all_images([ img[-1] for img in character_images],[name+'_refined' for name in character_names])
    return character_images


In [ ]:
character_images = refine_character_images(characters)

In [ ]:
character_images

In [ ]:
save_as_docx_few(characters,story,f'{story.page_title}_few_images_2.docx',suffix='_refined')

- translation

In [ ]:

SRC_LANG = 'English'
DIST_LANG = 'Chinese'


translate_chapter_prompt = ChatPromptTemplate.from_messages( 
 [    
   (
        "system",
        role_config["translator"]+fc_desc,
        ),
    MessagesPlaceholder(variable_name="messages", optional=True),  
 ])

async def translate_chapter(chapter_obj: DetailChapter,source_lang:str,dist_lang:str):
    name = "translator"
    translate_chapter_chain = translate_chapter_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=DetailChapter)
    request  = HumanMessage(content=f"Here is the content to be translated:\n{chapter_obj.json()}",name=name)
    translation = await retry_call(translate_chapter_chain,{"source_lang":source_lang,"dist_lang":dist_lang,"messages":[request],"schema":DetailChapter.schema_json()})
    return translation

In [ ]:
chaps = [ chatper for chatper in  story.chapters]
translate_all = RunnableLambda(translate_chapter).bind(source_lang=SRC_LANG,dist_lang=DIST_LANG)
all_chaps = await translate_all.abatch(chaps)
## merge all translated chapters
story_trans = Story(page_title=story.page_title,chapters=all_chaps)

In [ ]:
async def translate_character(characters_obj: Character,source_lang:str,dist_lang:str):
    name = "translator"
    translate_chapter_chain = translate_chapter_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=Character)
    request  = HumanMessage(content=f"Here is the content to be translated:\n{characters_obj.json()}",name=name)
    translation = await retry_call(translate_chapter_chain,{"source_lang":source_lang,"dist_lang":dist_lang,"messages":[request],"schema":Character.schema_json()})
    return translation

In [ ]:
characters_trans = await translate_character(characters,SRC_LANG,DIST_LANG)

In [ ]:
story_trans.images=images

In [ ]:
save_as_docx_few(characters_trans,story_trans,f'{story.page_title}_few_images_{DIST_LANG}.docx',suffix='_refined')

## 5. Concate all workflows

- Write outline

In [ ]:

async def create_outline_node(state:AgentState):
    steps = []
    async for event in outline_workflow.astream(state):
        steps.append(event)
        for key, value in event.items():
            print(f"Output from node '{key}':")
            print("---")
            print(value)
        print("\n---\n")
    env_var =  steps[-1]['generate_outline']['env_var']
    return {'env_var':{**env_var}}


- Write chapters

In [ ]:
async def create_chapters_node(state:AgentState):
    env_var = state['env_var']
    outline = env_var['outline']
    characters = env_var['characters']
    write_all = RunnableLambda(write_all_chapters).bind(outline=outline,characters=characters)
    chaps = [ chatper for chatper in  outline.chapters]
    # write chatpers in parallel
    all_steps =  await write_all.abatch(chaps)
    
    ## merge all English chapters
    story = Story(page_title=outline.page_title,chapters=[s[0] for s in all_steps])
    
    
    
    return {'env_var':{**env_var,"story":story}}
    

- Create Translation

In [ ]:

SRC_LANG = 'English'
DIST_LANG = 'Chinese'


translate_chapter_prompt = ChatPromptTemplate.from_messages( 
 [    
   (
        "system",
        role_config["translator"]+fc_desc,
        ),
    MessagesPlaceholder(variable_name="messages", optional=True),  
 ])

async def translate_chapter(chapter_obj: DetailChapter,source_lang:str,dist_lang:str):
    name = "translator"
    translate_chapter_chain = translate_chapter_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=DetailChapter)
    request  = HumanMessage(content=f"Here is the content to be translated:\n{chapter_obj.json()}",name=name)
    translation = await retry_call(translate_chapter_chain,{"source_lang":source_lang,"dist_lang":dist_lang,"messages":[request],"schema":DetailChapter.schema_json()})
    return translation

async def translate_character(characters_obj: Character,source_lang:str,dist_lang:str):
    name = "translator"
    translate_chapter_chain = translate_chapter_prompt | llm_sonnet | CustJsonOuputParser(verbose=True) |RunnableLambda(dict_to_obj).bind(target=Character)
    request  = HumanMessage(content=f"Here is the content to be translated:\n{characters_obj.json()}",name=name)
    translation = await retry_call(translate_chapter_chain,{"source_lang":source_lang,"dist_lang":dist_lang,"messages":[request],"schema":Character.schema_json()})
    return translation

async def translate_node(state: AgentState):
    env_var = state['env_var']
    story = env_var['story']
    assert not story == None
    chaps = [ chatper for chatper in  story.chapters]
    translate_all = RunnableLambda(translate_chapter).bind(source_lang=SRC_LANG,dist_lang=DIST_LANG)
    all_chaps = await translate_all.abatch(chaps)
    # merge all translated chapters
    story_trans = Story(page_title=story.page_title,chapters=all_chaps)
    # translate character description
    characters = env_var['characters']
    characters_trans = await translate_character(characters,SRC_LANG,DIST_LANG)
    
    return {'env_var':{**env_var,"story_trans":story_trans,"characters_trans":characters_trans}}
    
    



- Generate Images

In [ ]:

import uuid
from datetime import datetime
endpoint = 'story-diffusion-inference-api-2024-05-10-01-15-04-556'

# from sagemaker endpoint
stod_image_generator = StoryDiffusionGenerator(endpoint_name=endpoint,profile='4344')
image_generator = ImageGenerator(profile='4344')

gen_identity_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            role_config['art designer']+fc_desc,
        ),
        MessagesPlaceholder(variable_name="messages",optional=True),  
    ]
)

gen_image_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            role_config['story illustrator']
        ),
        MessagesPlaceholder(variable_name="messages",optional=True),  
    ]
)



def generate_all_identities(characters,gen_identity_chain):
    batch_inputs  = [{'messages':[HumanMessage(content=characters.main_character.appearance)],'schema':ImagePrompt.schema_json()}]
    character_names = [characters.main_character.name]
    for ch in characters.supporting_character:
        batch_inputs += [{'messages':[HumanMessage(content=ch.appearance)],'schema':ImagePrompt.schema_json()}]
        character_names += [ch.name]
    
    sd_prompts = gen_identity_chain.batch(batch_inputs)
    
    # use a self-hosted sdxl model to create portrait
    # images = [generate_images_sdxl_api(prompt=sd_prompt.prompt) for sd_prompt in sd_prompts]
    images = [image_generator.generate_image(prompt=sd_prompt.prompt) for sd_prompt in sd_prompts]
    # print(images)
    # save images in folder
    # fname = [datetime.now().strftime(f"%Y%m%d%H%M%S-{name}-{uuid.uuid4()}") for name in character_names]
    fname = save_all_images(images,character_names)
    return fname
        
async def generate_portrait_node(state:AgentState):
    env_var = state['env_var']
    characters = env_var['characters']
    gen_identity_chain = gen_identity_prompt | llm_sonnet | CustJsonOuputParser(verbose=False) |RunnableLambda(dict_to_obj).bind(target=ImagePrompt)    
    fnames = generate_all_identities(characters,gen_identity_chain)
    return {'env_var':{**env_var,"portrait_files":fnames}}

async def generate_images_node(state:AgentState):
    env_var = state['env_var']
    story = env_var['story']
    characters = env_var['characters']
    gen_image_prompt_chain =  gen_image_prompt | llm_sonnet | TextOuputParser(verbose=False)
    messages = []
    for chapter in story.chapters:
        messages += [{"messages":[HumanMessage(content='\n'.join([ p.content for p in chapter.paragraphs]))]}] 
    all_story_lines = await gen_image_prompt_chain.abatch(messages)
    
    #construct prompt for storydiffusion
    storyd_prompts,img_dicts = prepare_storyd_prompts(all_story_lines,characters)
    

    sytle = "Comic book"
    comic_type = "Four Pannel"
    images_list = []
    for i,p in enumerate(storyd_prompts):
        print(f"line:{i},{p['prompt_array']}")  
        images = stod_image_generator.generate_images(general_prompt = p['general_prompt'],
                                                        style=sytle,
                                                        comic_type = comic_type,
                                                        prompt_array='\n'.join(p['prompt_array']),
                                                        id_length=p['id_length'],
                                                        ref_imgs=p['ref_imgs'],height=512,width=768)
        images_list.append(images)
    ##reverse the output order, only use the combined image
    images = [img for imgs in images_list for img in imgs[::-1][-1:]] 

    # generate refined images for characters
    character_images = refine_character_images(characters)

    return {'env_var':{**env_var,"images":images,"all_story_lines":all_story_lines,
                       "images_list":images_list,"images":images,"character_images":character_images}}

- Generate final file

In [ ]:
async def generate_file_node(state:AgentState):
    env_var = state['env_var']
    story = env_var['story']
    story_trans = env_var['story_trans']
    characters = env_var['characters']
    characters_trans = env_var['characters_trans']
    images = env_var['images']
    story.images = images
    story_trans.images = images
    save_as_docx_few(characters,story,f'{story.page_title}.docx',suffix='_refined')
    save_as_docx_few(characters_trans,story_trans,f'{story_trans.page_title}_trans.docx',suffix='_refined')
    return {'env_var':{**env_var,"story":story,"story_trans":story_trans}}
    
    

In [ ]:
build_graph = StateGraph(AgentState)
build_graph.add_node("create_outline_node",create_outline_node)
build_graph.add_node("create_chapters_node",create_chapters_node)
build_graph.add_node("translate_node",translate_node)
build_graph.add_node("generate_portrait_node",generate_portrait_node)
build_graph.add_node("generate_images_node",generate_images_node)
build_graph.add_node("generate_file_node",generate_file_node)
build_graph.add_edge("create_outline_node","create_chapters_node")
build_graph.add_edge("create_chapters_node","generate_portrait_node")
build_graph.add_edge("generate_portrait_node","generate_images_node")
build_graph.add_edge("generate_images_node","translate_node")
build_graph.add_edge("translate_node","generate_file_node")
build_graph.set_entry_point("create_outline_node")
build_graph.set_finish_point("generate_file_node")
workflow = build_graph.compile()


In [ ]:

from IPython.display import Image

Image(workflow.get_graph().draw_png())


In [ ]:
topic = "续写一位霸道女总裁的故事。她从外交学院毕业之后，经过多年奋斗，终于进入了一家知名企业当副总裁，然而她对下属员工非常刻薄，她在个人抖音账号上发布了多条视频，包括“员工闹分手提离职我秒批”、“300封举报信撒满工位”等话题，这些视频内容引发了公众的热议和争议，终于被公司辞退。被公司辞退前她自己注册了一家公司。"

In [ ]:
topic="Continuing to write the story of a domineering female CEO. After graduating from the School of Foreign Affairs, she worked hard for many years and finally joined a well-known company as the vice president. However, she was very harsh towards her subordinates, which led to her being publicly posted online and reported, and finally dismissed by the company. Before being dismissed by the company, she registered a company herself."

In [ ]:
steps = []
init_state = {
    "env_var":{},
    "messages":[HumanMessage(content=f"Here is the topic:{topic}")]
}


In [ ]:
async for event in workflow.astream(input=init_state):
    steps.append(event)
    for key, value in event.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

In [ ]:
inter_state = AgentState(steps[-1]['generate_images_node'])

In [ ]:
translate_state = await translate_node(inter_state)

In [ ]:
await generate_file_node(translate_state)